# Etape 3 : Collecte des Données de Végétation pour la Prédiction des Incendies


## Contexte

Dans le cadre de notre projet de prédiction des incendies, nous utilisons des données de végétation fournies par la NASA, accessibles via la plateforme Earth Data. Ces données, issues du produit MODIS (Moderate Resolution Imaging Spectroradiometer), couvrent l'ensemble du globe et sont mises à jour tous les 16 jours. Chaque fichier de données correspond à une carte à la résolution spatiale de 500 mètres par pixel, offrant ainsi une vue détaillée de la végétation sur de vastes zones géographiques.


## Objectif de cette Étape

L'objectif de ce notebook est de rassembler et de préparer les données de végétation pour leur analyse dans le contexte de la prédiction des incendies. Nous procéderons par la manipulation de multiples fichiers HDF (Hierarchical Data Format) pour extraire les données pertinentes de végétation. Cette étape implique la consolidation des données issues de différentes sources et leur alignement avec des coordonnées géographiques et cartographiques précises. Le résultat sera un ensemble de données structuré et prêt à être intégré dans notre modèle prédictif, permettant ainsi d'évaluer l'impact de l'état de la végétation sur la probabilité des incendies.


### Utilisation de la bibliothèque pyhdf pour manipuler des fichiers HDF

In [1]:
!pip install pyhdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.6/771.6 kB 9.1 MB/s eta 0:00:00


In [2]:
!pip install pyproj

In [3]:
from pyproj import Proj, transform
import math
import pandas as pd 

## Sélection des Zones pour l'Extraction des Données de Végétation

Afin de préciser les zones d'intérêt d'où nous souhaitons extraire les données de végétation, nous définissons les coordonnées géographiques des coins inférieurs gauche et supérieur droite pour chaque zone sélectionnée. Ces coordonnées nous permettent de localiser n'importe quel point ayant ses données geographiques (longitude et latitude ) sur la carte mondiale.


In [4]:
   #zone 1
long_sw = -13.0257549017688
lat_sw =  39.8151379056937
long_ne =0.0216312604323565
lat_ne = 50.0061727485167
    #Zone2 
long_sw_ = -11.5363713866164
lat_sw_ =  29.8722651489315
long_ne_ = 0.0159719631030949
lat_ne_ =  40.0042974310271

In [5]:
file_181= "/kaggle/input/data-greece-7-2/MOD13A1.A2023193.h19v04.061.2023215122211.hdf"
file_281="/kaggle/input/data-greece-7-2/MOD13A1.A2023193.h19v05.061.2023215105058.hdf"
file_381="/kaggle/input/data-greece-7-2/MOD13A1.A2023193.h20v05.061.2023215105508.hdf"


## Manipulation des Fichiers HDF

Dans cette section, nous voyons comment manipuler les fichiers HDF, naviguer sur leurs datasets, leur architecture ainsi que les attributs de données.


In [6]:
from pyhdf.SD import SD, SDC

# Remplacer 'path/to/your/file.hdf' par le chemin réel vers le fichier HDF4 dans l'environnement Kaggle
file_path = "/kaggle/input/zone2-greece-8-2/MOD13A1.A2023225.h19v04.061.2023242000650.hdf"
file_path_2 = "/kaggle/input/data-greece-8-2/MOD13A1.A2023225.h19v05.061.2023242000526.hdf"


# Ouvrir le fichier HDF4
hdf = SD(file_path, SDC.READ)

# Afficher les datasets disponibles
datasets = hdf.datasets()
for ds in datasets:
    print(ds)



500m 16 days NDVI
500m 16 days EVI
500m 16 days VI Quality
500m 16 days red reflectance
500m 16 days NIR reflectance
500m 16 days blue reflectance
500m 16 days MIR reflectance
500m 16 days view zenith angle
500m 16 days sun zenith angle
500m 16 days relative azimuth angle
500m 16 days composite day of the year
500m 16 days pixel reliability


In [7]:
# Lire un dataset spécifique, par exemple 'dataset_name'
data = hdf.select('500m 16 days EVI').attributes()
data

{'long_name': '500m 16 days EVI',
 'units': 'EVI',
 'valid_range': [-2000, 10000],
 '_FillValue': -3000,
 'scale_factor': 10000.0,
 'scale_factor_err': 0.0,
 'add_offset': 0.0,
 'add_offset_err': 0.0,
 'calibrated_nt': 5}

In [8]:
data = hdf.select('500m 16 days EVI')[:]
len(data)

2400

In [9]:
data[1215]

array([-3000, -3000, -3000, ...,  3311,  3262,  2961], dtype=int16)

## Récupération de la Position d'un Point de Coordonnées sur la Grille de Données

Dans cette section, nous allons voir comment récupérer la position d'un point de coordonnées latitude et longitude sur une grille de données de longueur et hauteur 2400.

### Système de Projection Sinusoïdale

Ce produit utilise un système de projection sinusoïdale. Pour convertir les données géographiques aux coordonnées cartographiques dont l'origine est à l'équateur, nous utilisons les formules suivantes :

lambda = lambda * cos(phi)
phi = phi 

où :
- lambda est la longitude
- hi est la latitude

Les angles doivent être exprimés en radians.

### Conversion et Positionnement

Étant donné cette formule, il sera facile de positionner n'importe quel point sur la grille en utilisant sa résolution. Cela nous permet de récupérer l'information disponible dans la grille à la position correspondante.

### Exemple de Conversion en Python
Ensuite, nous pouvons utiliser ce principe pour convertir des coordonnées géographiques en coordonnées cartographiques et les utiliser pour récupérer des données de la grille.



In [10]:
def calculate_Grid_Position(long,lat, lat_ne, lat_sw, long_ne , long_sw):
    #zone 1
    long_sw = -13.0257549017688
    lat_sw =  39.8151379056937
    long_ne =0.0216312604323565
    lat_ne = 50.0061727485167
    #Zone2 
    long_sw_ = -11.5363713866164
    lat_sw_ =  29.8722651489315
    long_ne_ = 0.0159719631030949
    lat_ne_ =  40.0042974310271
    

    
    if lat >=lat_sw  :
        lat_ne, lat_sw, long_ne , long_sw = lat_ne, lat_sw, long_ne , long_sw
    else :
        lat_sw, long_ne , long_sw, lat_ne = lat_sw_, long_ne_ , long_sw_, lat_ne_
    x0 = long_sw * math.pi /180 * math.cos(lat_sw * math.pi /180)
    y0 = lat_sw  * math.pi /180
    x1 = long_ne * math.pi /180 * math.cos(lat_ne * math.pi /180)
    y1 = lat_ne  * math.pi /180
    x_ = long * math.pi /180 * math.cos(lat * math.pi /180) 
    y_ = lat  * math.pi /180
    delta_x = x_-x0
    delta_y = y_-y0
    x = delta_x * 2399 / (x1-x0)
    y = delta_y * 2399 / (y1-y0)
    y = 2399 - y
    
    
    if x>=0 and x<=2399 and  y>=0 and y<=2399 :
        return round(y), round(x)
        
    return "not_found"
     
    

    
    
    

In [11]:
#mois 2 
data = pd.read_csv('/kaggle/input/dataaa/data_Portugal_-Copy1.csv')

In [12]:
data

,latitude,longitude,acq_date,acq_time,class
0,39.8053,-8.6446,2023-08-03,1408,not fire
1,40.9737,-7.9931,2023-02-13,1354,not fire
2,37.4558,-8.6925,2023-08-07,1425,fire
3,39.7427,-7.7687,2023-08-05,1035,fire
4,39.7274,-7.7657,2023-08-05,1303,fire
...,...,...,...,...,...
1071,37.3930,-8.6284,2023-08-07,2122,fire
1072,39.7776,-7.7632,2023-08-05,1035,fire
1073,37.4010,-8.7115,2023-08-06,2219,fire
1074,37.4002,-8.6646,2023-08-27,1425,not fire


In [13]:
data['acq_date'] = pd.to_datetime(data['acq_date'])

## Filtrage et Rassemblement des Données par Périodes de 16 Jours

Étant donné que les informations sont associées à des périodes de 16 jours, nous allons filtrer les données pour chaque période et rassembler les données pour chaque point de notre data. 



In [14]:
filtered_data = data[(data['acq_date'] >= '2023-02-02') & (data['acq_date'] <= '2023-02-17')]

In [15]:
filtered_data

,latitude,longitude,acq_date,acq_time,class
1,40.9737,-7.9931,2023-02-13,1354,not fire
21,41.1871,-7.9548,2023-02-17,1113,not fire
26,40.9948,-8.0812,2023-02-02,1122,not fire
36,40.9996,-8.0012,2023-02-11,1407,not fire
37,41.6879,-7.9059,2023-02-10,1356,fire
...,...,...,...,...,...
1043,41.7945,-7.6171,2023-02-08,1303,not fire
1047,41.8782,-7.1245,2023-02-07,1359,not fire
1051,38.6692,-8.8861,2023-02-06,1346,not fire
1056,41.6576,-7.7840,2023-02-14,2153,not fire


In [16]:
def get_zone(lat,long):
    lat_sw =  29.8722651489315
    if lat >=lat_sw : 
        return "/kaggle/input/data-po-8-2/MOD13A1.A2023225.h17v04.061.2023242000947 (1).hdf"
    
    return "/kaggle/input/data-po-8-2/MOD13A1.A2023225.h17v05.061.2023242000656 (1).hdf"
   

    
def get_pixel_value_from_hdf(lat, long):
       #zone 1
        long_sw = -13.0257549017688
        lat_sw =  39.8151379056937
        long_ne =0.0216312604323565
        lat_ne = 50.0061727485167
        #Zone2 
        long_sw_ = -11.5363713866164
        lat_sw_ =  29.8722651489315
        long_ne_ = 0.0159719631030949
        lat_ne_ =  40.0042974310271
    
        file_path = get_zone(lat,long)
        if file_path =="/kaggle/input/data-po-8-2/MOD13A1.A2023225.h17v04.061.2023242000947 (1).hdf" : 
            x, y = calculate_Grid_Position(long,lat,lat_ne, lat_sw, long_ne , long_sw)
        elif file_path ==  "/kaggle/input/data-po-8-2/MOD13A1.A2023225.h17v05.061.2023242000656 (1).hdf" : 
            x, y = calculate_Grid_Position(long,lat,lat_ne_, lat_sw_, long_ne_ , long_sw_)

            
            
         
        hdf = SD(file_path, SDC.READ)
        data = hdf.select("500m 16 days NDVI")[:]
        NDVI = data[x][y] /10000
        data2 = hdf.select("500m 16 days EVI")[:]
        EVI = data2[x][y] /10000
        results = {"NDVI": NDVI , "EVI" : EVI}
        return results
def apply_vegetation_data(row):
    try:
        
    
        # Appeler fetch_weather_data pour chaque ligne
        results = get_pixel_value_from_hdf(row['latitude'],row['longitude'])
        return pd.Series(results)
    except Exception as e:
        print(f"Erreur lors de la récupération des données météo: {e}")
        return pd.Series({key: np.nan for key in results.keys()})


def add_data_column(df, dataset):
    # Apply the function to longitude and latitude columns
    df[dataset] = df.apply(get_pixel_value_from_hdf(row['latitude'], row['longitude'], dataset), axis=1)




In [17]:
# Appliquer la fonction pour chaque ligne et récupérer les résultats
vegetation_metrics = filtered_data.apply(apply_vegetation_data, axis=1)

# Ajouter les résultats comme nouvelles colonnes dans le DataFrame original
filtered_data = pd.concat([filtered_data, vegetation_metrics], axis=1)

In [18]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 1 to 1060
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   latitude   101 non-null    float64       
 1   longitude  101 non-null    float64       
 2   acq_date   101 non-null    datetime64[ns]
 3   acq_time   101 non-null    int64         
 4   class      101 non-null    object        
 5   NDVI       101 non-null    float64       
 6   EVI        101 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 6.3+ KB


## Suppression des Valeurs Nulles

En consultant les attributs des données, nous avons identifié que les valeurs -0.3 correspondent aux valeurs nulles. Alors nous allons supprimer ces valeurs nulles des données de végétation.



In [19]:
filtered_data = filtered_data[filtered_data["NDVI"] != -0.3]

In [20]:
filtered_data.to_csv("vegetation_data_2_1.csv", index=False)

In [21]:
filtered_data = data[(data['acq_date'] >= '2023-02-18') & (data['acq_date'] <= '2023-02-28')]

In [22]:
# Appliquer la fonction pour chaque ligne et récupérer les résultats
vegetation_metrics = filtered_data.apply(apply_vegetation_data, axis=1)

# Ajouter les résultats comme nouvelles colonnes dans le DataFrame original
filtered_data = pd.concat([filtered_data, vegetation_metrics], axis=1)

In [23]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69 entries, 12 to 1068
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   latitude   69 non-null     float64       
 1   longitude  69 non-null     float64       
 2   acq_date   69 non-null     datetime64[ns]
 3   acq_time   69 non-null     int64         
 4   class      69 non-null     object        
 5   NDVI       69 non-null     float64       
 6   EVI        69 non-null     float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 4.3+ KB


In [24]:
filtered_data =filtered_data[filtered_data["EVI"] != -0.3]

In [25]:
filtered_data

,latitude,longitude,acq_date,acq_time,class,NDVI,EVI
12,41.6684,-8.1165,2023-02-24,2211,not fire,0.7134,0.3773
67,41.6970,-7.8540,2023-02-25,2234,not fire,0.7348,0.3819
77,40.9942,-7.9938,2023-02-21,1407,not fire,0.5845,0.3299
83,41.7675,-7.9042,2023-02-19,1345,fire,0.3964,0.1623
102,41.6576,-7.7840,2023-02-24,2153,not fire,0.8119,0.4178
...,...,...,...,...,...,...,...
999,41.8647,-7.9314,2023-02-23,1354,not fire,0.7447,0.4384
1014,40.9737,-7.9931,2023-02-23,1354,not fire,0.6275,0.3417
1038,41.7324,-8.3500,2023-02-25,235,not fire,0.7350,0.3503
1048,40.9948,-8.0812,2023-02-22,1122,not fire,0.4605,0.2514


In [26]:
filtered_data.to_csv("vegetation_data_2_2.csv", index=False)

In [27]:
filtered_data = data[(data['acq_date'] >= '2023-08-01') & (data['acq_date'] <= '2023-08-12')]

In [28]:
# Appliquer la fonction pour chaque ligne et récupérer les résultats
vegetation_metrics = filtered_data.apply(apply_vegetation_data, axis=1)

# Ajouter les résultats comme nouvelles colonnes dans le DataFrame original
filtered_data = pd.concat([filtered_data, vegetation_metrics], axis=1)

In [29]:
 filtered_data =  filtered_data[ filtered_data["EVI"] != -0.3]

In [30]:
filtered_data.to_csv("vegetation_data_8_1.csv", index=False)

In [31]:
filtered_data = data[(data['acq_date'] >='2023-08-13') & (data['acq_date'] <= '2023-08-31')]

In [32]:
# Appliquer la fonction pour chaque ligne et récupérer les résultats
vegetation_metrics = filtered_data.apply(apply_vegetation_data, axis=1)

# Ajouter les résultats comme nouvelles colonnes dans le DataFrame original
filtered_data = pd.concat([filtered_data, vegetation_metrics], axis=1)

In [33]:
filtered_data[ filtered_data["EVI"] != -0.3]

,latitude,longitude,acq_date,acq_time,class,NDVI,EVI
24,41.6516,-6.5686,2023-08-30,2147,not fire,0.5337,0.2829
25,41.1825,-7.9013,2023-08-21,2243,fire,0.6891,0.3093
31,41.3207,-7.3189,2023-08-14,2114,fire,0.5715,0.2834
40,40.5304,-7.8067,2023-08-23,2226,fire,0.5819,0.3814
58,41.6897,-8.2084,2023-08-16,305,fire,0.7160,0.3573
...,...,...,...,...,...,...,...
1034,41.6946,-7.7698,2023-08-20,2218,not fire,0.5578,0.2945
1040,40.5337,-7.8422,2023-08-23,1409,fire,0.3252,0.2427
1053,40.8502,-7.3100,2023-08-25,1354,fire,0.5962,0.3209
1061,41.6897,-8.2084,2023-08-26,305,not fire,0.7160,0.3573


In [34]:
filtered_data.to_csv("vegetation_data_8_2.csv", index=False)

In [35]:
df1 = pd.read_csv("/kaggle/working/vegetation_data_2_1.csv")
df2 = pd.read_csv("/kaggle/working/vegetation_data_2_2.csv")
df3 = pd.read_csv("/kaggle/working/vegetation_data_8_1.csv")
df4 = pd.read_csv("/kaggle/working/vegetation_data_8_2.csv")


In [36]:
concatenated_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [37]:
shuffled_df = concatenated_df.sample(frac=1).reset_index(drop=True)

In [38]:
shuffled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   806 non-null    float64
 1   longitude  806 non-null    float64
 2   acq_date   806 non-null    object 
 3   acq_time   806 non-null    int64  
 4   class      806 non-null    object 
 5   NDVI       806 non-null    float64
 6   EVI        806 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 44.2+ KB


In [39]:
shuffled_df.to_csv("vegetation_data_Portugal.csv" , index= False )

In [40]:

data = pd.read_csv('/kaggle/input/data-greece/data_Greece_-Copy1.csv')

In [41]:
data['acq_date'] = pd.to_datetime(data['acq_date'])

In [42]:
filtered_data = data[(data['acq_date'] >= '2023-08-13') & (data['acq_date'] <= '2023-08-31')]

In [43]:
filtered_data

,latitude,longitude,acq_date,acq_time,class
0,40.8753,25.7921,2023-08-22,43,fire
5,40.8845,26.0477,2023-08-19,1943,fire
7,36.0252,27.8657,2023-08-14,2040,not fire
8,38.4728,26.0022,2023-08-20,6,not fire
10,40.9083,25.8666,2023-08-31,1926,not fire
...,...,...,...,...,...
7151,36.0873,27.9160,2023-08-14,1902,not fire
7157,41.0555,26.2117,2023-08-16,1119,not fire
7159,41.0349,26.0893,2023-08-21,1926,fire
7160,41.0099,25.9632,2023-08-21,1926,fire


In [ ]:

 #zone 1
long_sw =  12.9805303891434
lat_sw =  39.8218653179575
long_ne =31.1317532002432
lat_ne =  49.9988111339122
#Zone2 
long_sw_ =  11.4946633097324
lat_sw_ =  29.8782895599206
long_ne_ =  26.1216095695463
lat_ne_ =  39.9992246349507

In [ ]:
calculate_Grid_Position(27.8657,36.0252, lat_ne, lat_sw, long_ne , long_sw)

In [ ]:
 filtered_data[ filtered_data["EVI"] == -0.3].info()

In [ ]:
filtered_data[ filtered_data["EVI"] != -0.3].info()

In [ ]:
filtered_data.to_csv("vegetation_greece_8_2.csv", index=False)

In [ ]:
filtered_data = data[(data['acq_date'] >= '2023-07-28') & (data['acq_date'] <= '2023-08-12')]

In [ ]:
# Appliquer la fonction pour chaque ligne et récupérer les résultats
vegetation_metrics = filtered_data.apply(apply_vegetation_data, axis=1)

# Ajouter les résultats comme nouvelles colonnes dans le DataFrame original
filtered_data = pd.concat([filtered_data, vegetation_metrics], axis=1)

In [ ]:
filtered_data[ filtered_data["EVI"] != -0.3].info()

In [ ]:
filtered_data.to_csv("vegetation_greece_8_1.csv", index=False)

In [ ]:
filtered_data = data[(data['acq_date'] >= '2023-07-12') & (data['acq_date'] <= '2023-07-27')]

In [ ]:
# Appliquer la fonction pour chaque ligne et récupérer les résultats
vegetation_metrics = filtered_data.apply(apply_vegetation_data, axis=1)

# Ajouter les résultats comme nouvelles colonnes dans le DataFrame original
filtered_data = pd.concat([filtered_data, vegetation_metrics], axis=1)

In [ ]:
filtered_data[ filtered_data["EVI"] != -0.3].info()

In [ ]:
filtered_data.to_csv("vegetation_greece_7_2.csv", index=False)

In [ ]:
filtered_data = data[(data['acq_date'] >= '2023-07-01') & (data['acq_date'] <= '2023-07-11')]

In [ ]:
# Appliquer la fonction pour chaque ligne et récupérer les résultats
vegetation_metrics = filtered_data.apply(apply_vegetation_data, axis=1)

# Ajouter les résultats comme nouvelles colonnes dans le DataFrame original
filtered_data = pd.concat([filtered_data, vegetation_metrics], axis=1)

In [ ]:
filtered_data[ filtered_data["EVI"] != -0.3].info()

In [ ]:
filtered_data.to_csv("vegetation_greece_7_1.csv", index=False)

In [ ]:
df1 = pd.read_csv("/kaggle/working/vegetation_greece_7_1.csv")
df2 = pd.read_csv("/kaggle/working/vegetation_greece_7_2.csv")
df3 = pd.read_csv("/kaggle/working/vegetation_greece_8_1.csv")
df4 = pd.read_csv("/kaggle/working/vegetation_greece_8_2.csv")

In [ ]:
concatenated_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [ ]:
shuffled_df = concatenated_df.sample(frac=1).reset_index(drop=True)

In [ ]:
shuffled_df.to_csv("vegetation_data_greece.csv")